In [ ]:
# !pip install simplemma

In [ ]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

In [ ]:
stop_words = None
with open("stopwords.txt", "r") as stop_file:
    stop_words = set(stop_file.read().splitlines())

In [ ]:
stop_words

In [ ]:
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)

    text = " ".join(text.lower().strip().split())
    text = text_lemmatizer(text, lang="tr")

    return " ".join([word for word in text if word not in stop_words])

In [ ]:
cumle = "Hey! Naber???İyilik, senden... Ben de iyiyim.Çocuk filmi Kirpi Sonic filminin devam halkası olan yapımda, insan biçiminde bir kirpi olan Sonic'in atıldığı maceralar konu ediliyor. Green Hills’e yerleşen Sonic’in tek isteği, gerçek bir kahraman olması için gereken özelliklere sahip olduğunu kanıtlayabilmektir. Sonic’in sınavı, Dr. Robotnik’in ortağı Knuckes ile birlikte medeniyetleri yok edebilecek güce sahip bir zümrüdü aramak için geri dönmesiyle başlar. Zümrüdün yanlış ellere geçmesini engellemek için Sonic, yardımcısı Tails ile birlikte macera dolu bir yolculuğa çıkar."
clean_text(cumle)

In [ ]:
df = pd.read_excel("filmveriseti.xlsx") 
# https://www.kaggle.com/code/furkanyeter/content-based-recommendation-system-turkish-movie/data
df.head()

In [ ]:
len(df)

In [ ]:
df["tur"]

In [ ]:
df.iloc[4]["ozet ve detaylar"]

In [ ]:
df.iloc[4]["tur"]

In [ ]:
type(df.iloc[4]["tur"])

In [ ]:
df["tur_list"] = df["tur"].apply(ast.literal_eval)

In [ ]:
df.iloc[4]["tur_list"]

In [ ]:
def clean_row(row):
    ozet = clean_text(row["ozet ve detaylar"])
    tur = row["tur_list"]
    return ozet + " " + " ".join(tur)

In [ ]:
clean_row(df.iloc[1])

In [ ]:
df["clean"] = df.apply(lambda row: clean_row(row), axis=1) 

In [ ]:
df.head()

In [ ]:
import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["ozet ve detaylar"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(df.iloc[rand_idx]["clean"])
    print("/" * 50 + "\n")  

Sözlük oluşturma başlıyor

In [ ]:
kelime2index = {}
index2kelime = {}

In [ ]:
kelime2index["<UNK>"] = 0
index2kelime[0] = "<UNK>"

In [ ]:
clean_col = df["clean"]
clean_col

In [ ]:
clean_col.values

In [ ]:
i = 1
for doc in clean_col.values:
    for token in doc.split():
        if token in kelime2index:
            continue
        
        kelime2index[token] = i
        index2kelime[i] = token
        i += 1

In [ ]:
kelime2index

In [ ]:
index2kelime

In [ ]:
len(kelime2index)

Sayma vektörleri (Counting vectors)

In [ ]:
doc_mat = np.zeros((len(df), len(kelime2index)))

In [ ]:
for i, doc in enumerate(clean_col.values):
    for token in doc.split():
        token_index = kelime2index[token]
        doc_mat[i][token_index] += 1

In [ ]:
doc_mat[0]

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

Sorgular

In [ ]:
random.seed(42)
film_indexi = random.randrange(len(df))
film = df.iloc[film_indexi]
film["film adi"], film["ozet ve detaylar"], film["tur"]

In [ ]:
benzerlikler = []
sorgu_film = doc_mat[film_indexi]
for vec in doc_mat:
    benzerlik = cos_sim(sorgu_film, vec)
    benzerlikler.append(benzerlik)

In [ ]:
test_list = [1, 5, 4, 2]
print(np.array(test_list).argsort()[:2])
print((- np.array(test_list)).argsort()[:2])

In [ ]:
# en yakın 5 film
top5_film_indexleri = (- np.array(benzerlikler)).argsort()[:5]

In [ ]:
for i in top5_film_indexleri:
    film = df.iloc[i]
    print(film["film adi"], film["ozet ve detaylar"], film["tur"])
    print(" - " * 20)

In [ ]:
def print_film(row_index):
    film = df.iloc[row_index]
    adi = film["film adi"]
    turu = ", ".join(film["tur_list"])
    ozet = film["ozet ve detaylar"]
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    print(f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}")

In [ ]:
def benzerlerini_bul(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print_film(film_indexi)
    print(" * " * 20)
    print(" * " * 20)

    benzerlikler = []
    sorgu_film = doc_mat[film_indexi]
    for vec in doc_mat:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)
    
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i in en_benzer_film_indexleri[1:]:
        print_film(i)
        print(" - " * 20)
        print(" - " * 20)

In [ ]:
benzerlerini_bul("Kirpi Sonic 2", 3)